## Exercise 2: Chemical Equation Balancer Checker

**Objective:** Write a program that checks if a chemical equation is balanced.

**Problem:** Create a function called `is_balanced()` that takes a chemical equation string and returns True if balanced, False if not.

**Your Task:**
- Split the equation at the arrow symbol "→" to get reactants and products
- For each side, parse the molecules and their coefficients
- Count total atoms of each element on both sides
- Compare the totals

Example Equation Format: "2H2 + O2 → 2H2O"

**Hints:**

- Use equation.split("→") to separate reactants from products
- Use side.split(" + ") to separate individual molecules
- You'll need to handle coefficients (numbers in front of molecules)
- Reuse your count_atoms() function from Exercise 1!
- For molecules with coefficients like "2H2O", the coefficient applies to all atoms in that molecule

**Steps to Think Through:**

- How do you extract the coefficient from "2H2O"? (It's everything before the first letter)
- How do you multiply the atom counts by the coefficient?
- How do you combine counts from multiple molecules on the same side?


# Lecture 2: Computational Thinking with Python

In [26]:
from typing import Dict, List, Tuple
import re
from collections import defaultdict

def parse_molecular_formula(formula: str) -> Dict[str, int]:
    """
    Parse a formula like "C2H5OH" -> {"C":2, "H":6, "O":1}
    """
    pattern = r"([A-Z][a-z]?)([0-9]*)"
    components = re.findall(pattern, formula)
    atom_dict: Dict[str, int] = defaultdict(int)
    for name, num_str in components:
        count = int(num_str) if num_str else 1
        atom_dict[name] += count
    return dict(atom_dict)

def parse_molecule_with_coeff(molecule: str) -> Tuple[int, Dict[str, int]]:
    """
    Parse a molecule possibly with a leading coefficient, e.g. "2H2O" -> (2, {"H":2,"O":1})
    """
    molecule = molecule.strip()
    m = re.match(r"^(\d*)(.*)$", molecule)
    if not m:
        return 1, {}
    coeff_str, formula = m.groups()
    coeff = int(coeff_str) if coeff_str else 1
    formula = formula.strip()
    return coeff, parse_molecular_formula(formula)

def is_balanced(chem_eq: str) -> bool:
    # accept both '→' and '->'
    chem_eq = chem_eq.replace("->", "→")
    left_side, right_side = chem_eq.split("→")
    def total_counts(side: str) -> Dict[str, int]:
        totals: Dict[str, int] = defaultdict(int)
        for mol in side.split("+"):
            mol = mol.strip()
            if not mol:
                continue
            coeff, atoms = parse_molecule_with_coeff(mol)
            for el, n in atoms.items():
                totals[el] += coeff * n
        return dict(totals)

    return total_counts(left_side) == total_counts(right_side)


### Test Cases


In [28]:
# If the outcome is incorrect, the keyword assert will cause an error

print(is_balanced("2H2 + O2 → 2H2O"))
assert is_balanced("2H2 + O2 → 2H2O")

print(is_balanced("H2 + O2 → H2O"))
assert not is_balanced("H2 + O2 → H2O")

print(is_balanced("CH4 + 2O2 → CO2 + 2H2O"))
assert is_balanced("CH4 + 2O2 → CO2 + 2H2O")

print(is_balanced("N2 + H2 → NH3"))
assert not is_balanced("N2 + H2 → NH3")

True
False
True
False


## Exercise 3: Reaction Pathway Analyzer

**Objective:** Analyze multi-step reaction sequences to find synthesis pathways.

**Problem:** Write a program that determines what compounds can be synthesized from starting materials through a series of reactions, and finds a pathway to create a target compound.

**Your Task:**
Create a function called `find_synthesis_path()` that takes:

- A list of starting compounds
- A list of reaction equations
- A target compound to synthesize

The function should return either:

- A list of reaction steps that lead to the target compound, OR
- A message saying the target cannot be synthesized

**Hints:**

- Start by creating a set of "available" compounds (your starting materials)
- For each reaction, check if you have all the reactants needed
- If you can perform a reaction, add the products to your available compounds
- Keep track of which reactions you've used in what order
- Continue until you either find the target or can't make any new compounds

#### Computational Thinking:

**Decomposition:** Break this into smaller problems

- Parse reaction equations
- Track available compounds
- Check if reactions can be performed
- Record the synthesis pathway


**Pattern Recognition:** What patterns do you see?

- Each reaction has the same format: "reactants → products"
- You need to repeatedly check if new reactions become possible


**Algorithm Design:** What's your step-by-step approach?

- Initialize available compounds
- Loop: try each reaction, see if possible, update available compounds
- Stop when target is found or no progress is made


**Abstraction:** What functions might be helpful?

- A function to parse a single reaction equation
- A function to check if a reaction is possible with current compounds
- A function to update available compounds after a reaction



In [48]:
from typing import List, Tuple, Set

def find_synthesis_path(starting_materials: List[str], reactions: List[str], target: str) -> List[str] | str:

    # Write your code here
    #iterative approach: we check which reaction has target on the right side, then we check if we can make the left side from starting materials, if not we check which reaction has the left side as a result and so on...
    list_of_steps = [] #everytime we use a reaction we add it to this list
    available_materials = set(starting_materials) #we keep track of the available materials, we get more as we use reactions
    def parse_side(side: str) -> List[str]:
        return [m.strip() for m in side.split("+") if m.strip()]

    def parse_reaction(r: str) -> Tuple[Set[str], List[str], str]:
        """
        Parse a reaction string like "2H2 + O2 → 2H2O" and return:
         - reactants: set of reactant formulas without leading coefficients (e.g. {"H2","O2"})
         - products: list of product formulas without leading coefficients (e.g. ["H2O"])
         - original reaction string (trimmed)
        """
        def strip_coeff(mol: str) -> str:
            # remove leading digits and whitespace (e.g. "2H2" -> "H2", " 3 CO" -> "CO")
            mol = mol.strip()
            i = 0
            while i < len(mol) and (mol[i].isdigit() or mol[i].isspace()):
                i += 1
            return mol[i:].strip()

        left, right = r.split("→")
        reactants = set(strip_coeff(m) for m in parse_side(left))
        products = [strip_coeff(m) for m in parse_side(right)]
        return reactants, products, r.strip()
    
    def reaction_can_make(reaction: str, product: str) -> bool:
        reactants, products, r = parse_reaction(reaction)
        return product in products
    for i in range(10): #so its not infinite loop, we limit to 10 iterations
        for reaction in reactions:
            if reaction_can_make(reaction, target):
                reactants, products, r = parse_reaction(reaction)
                if all(reactant in available_materials for reactant in reactants):
                    list_of_steps.append(reaction)
                    available_materials.update(products)
                    if target in available_materials:
                        return list_of_steps
                else:
                    for reactant in reactants:
                        if reactant not in available_materials:
                            # recursively try to make the reactant
                            sub_path = find_synthesis_path(list(available_materials), reactions, reactant)
                            if sub_path == "Not possible":
                                continue
                            list_of_steps.extend(sub_path)
                            available_materials.add(reactant)
    return list_of_steps if target in available_materials else "Not possible"

### Test Cases

In [49]:
# Basic Multi-Step Synthesis

starting_materials = ["H2", "O2", "N2"]
reactions = [
    "2H2 + O2 → 2H2O",
    "N2 + 3H2 → 2NH3", 
    "NH3 + H2O → NH4OH"
]
target = "NH4OH"

result = find_synthesis_path(starting_materials, reactions, target)
print(result)
# Expected: Should find a path using H2 + N2 → NH3, then NH3 + H2O → NH4OH

['2H2 + O2 → 2H2O', 'N2 + 3H2 → 2NH3', 'NH3 + H2O → NH4OH']


In [50]:
# Direct Synthesis (One Step)

starting_materials = ["Na", "Cl2"]
reactions = [
    "2Na + Cl2 → 2NaCl",
    "NaCl + H2O → NaOH + HCl"
]
target = "NaCl"

result = find_synthesis_path(starting_materials, reactions, target)
print(result)
# Expected: Should find direct synthesis in one step

['2Na + Cl2 → 2NaCl']


In [51]:
# Impossible Synthesis

starting_materials = ["H2", "O2"]
reactions = [
    "2H2 + O2 → 2H2O",
    "N2 + 3H2 → 2NH3"  # But we don't have N2!
]
target = "NH3"

result = find_synthesis_path(starting_materials, reactions, target)
print(result)
# Expected: Should return message that NH3 cannot be synthesized

Not possible


In [52]:
# Multiple Pathway Options
starting_materials = ["C", "H2", "O2", "H2O"]
reactions = [
    "C + O2 → CO2",
    "2H2 + O2 → 2H2O",
    "CO2 + H2O → H2CO3",
    "C + 2H2 → CH4",
    "CH4 + 2O2 → CO2 + 2H2O"
]
target = "CO2"

result = find_synthesis_path(starting_materials, reactions, target)
print(result)
# Expected: Should find one of multiple possible paths to CO2

['C + O2 → CO2']


In [53]:
# Long Chain Synthesis
starting_materials = ["Fe", "O2", "C"]
reactions = [
    "4Fe + 3O2 → 2Fe2O3",
    "2C + O2 → 2CO",
    "Fe2O3 + 3CO → 2Fe + 3CO2",
    "CO2 + C → 2CO"
]
target = "CO2"

result = find_synthesis_path(starting_materials, reactions, target)
print(result)
# Expected: Should find the multi-step path involving iron oxide formation and reduction

['2C + O2 → 2CO', '4Fe + 3O2 → 2Fe2O3', 'Fe2O3 + 3CO → 2Fe + 3CO2']
